In [1]:
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urlencode
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import requests
import argparse
import hashlib
import pickle
import nltk
import re
import os

In [2]:
# nltk.download('stopwords')

# CACHE_DIR = os.path.join(os.path.abspath(os.path.dirname(__file__)), 'cache')
API_BASE_URL = 'https://api.hh.ru/vacancies'

DEFAULT_PARAMETERS = {
    'area': 1624, #kazan
    'per_page': 50,
}

HH_URL = API_BASE_URL + '?' + urlencode(DEFAULT_PARAMETERS)

EX_URL = 'https://api.exchangerate-api.com/v4/latest/RUB'
MAX_WORKERS = int(os.getenv('MAX_WORKERS', 5))

exchange_rates = {}

In [3]:
def update_exchange_rates():
    """
    Parse exchange rate for RUB, USD, EUR and save them to `exchange_rates`
    """
    try:
        print('Try to get rates from URL...')
        resp = requests.get(EX_URL)
        rates = resp.json()['rates']

    except requests.exceptions.SSLError:
        print('Cannot get exchange rate! Try later or change host API')
        exit('Exit from script. Cannot get data from URL!')

    for curr in ['RUB', 'USD', 'EUR']:
        exchange_rates[curr] = rates[curr]

    # Change 'RUB' to 'RUR'
    exchange_rates['RUR'] = exchange_rates.pop('RUB')
    print(f'Get exchange rates: {exchange_rates}')

In [4]:
def clean_tags(str_html):
    """
    Remove HTML tags from string (text)

    Parameters
    ----------
    str_html: str
        Input string with tags

    Returns
    -------
    string
        Clean text without tags

    """
    pat = re.compile('<.*?>')
    res = re.sub(pat, '', str_html)
    return res

In [5]:
def get_vacancy(vacancy_id):
    # Vacancy URL
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    vacancy = requests.api.get(url).json()

    # Extract salary
    salary = vacancy['salary']

    # Calculate salary:
    # Get salary into {RUB, USD, EUR} with {Gross} parameter and
    # return a new salary in RUB.
    cl_ex = {'from': None, 'to': None}
    if salary:
        # fn_gr = lambda: 0.87 if vsal['gross'] else 1
        def fn_gr():
            return 0.87 if vacancy['salary']['gross'] else 1

        for i in cl_ex:
            if vacancy['salary'][i] is not None:
                cl_ex[i] = int(
                    fn_gr() * salary[i] / exchange_rates[salary['currency']]
                )

    # Create pages tuple
    return (
        vacancy_id,
        vacancy['employer']['name'],
        vacancy['name'],
        salary is not None,
        cl_ex['from'],
        cl_ex['to'],
        vacancy['experience']['name'],
        vacancy['schedule']['name'],
        [el['name'] for el in vacancy['key_skills']],
        clean_tags(vacancy['description']),
    )

In [6]:
def get_vacancies(query, refresh=False):
    """
    Parse vacancy JSON: get vacancy name, salary, experience etc.

    Parameters
    ----------
    query: str
        Search query
    refresh: bool
        Refresh cached data

    Returns
    -------
    list
        List of useful arguments from vacancies

    """
#     cache_hash = hashlib.md5(query.encode()).hexdigest()
#     cache_file_name = os.path.join(CACHE_DIR, cache_hash)
#     try:
#         if not refresh:
#             return pickle.load(open(cache_file_name, 'rb'))

#     except (FileNotFoundError, pickle.UnpicklingError):
#         pass

    ids = []
    parameters = {'text': query, **DEFAULT_PARAMETERS}
    url = API_BASE_URL + '?' + urlencode(parameters)
    nm_pages = requests.get(url).json()['pages']
    for i in range(nm_pages + 1):
        resp = requests.get(url, {'page': i})
        data = resp.json()
        if 'items' not in data:
            break
        ids.extend(x['id'] for x in data['items'])

    vacancies = []
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        for vacancy in tqdm(executor.map(get_vacancy, ids), total=len(ids)):
            vacancies.append(vacancy)

    pickle.dump(vacancies, open('hh_data.csv', 'wb'))
    return vacancies

In [7]:
def prepare_df(dct_df):
    """
    Prepare data frame and save results

    Parameters
    ----------
    dct_df: list
        List of parsed json dicts

    """
    # List of columns
    df_cols = ['Id',
               'Employer',
               'Name',
               'Salary',
               'From',
               'To',
               'Experience',
               'Schedule',
               'Keys',
               'Description',
               ]
    # Create pandas dataframe
    df = pd.DataFrame(data=dct_df, columns=df_cols)
    # Print some info from data frame
#     print(
#         df[
#             df['Salary']
#         ]
#         [
#             ['Employer', 'From', 'To', 'Experience', 'Schedule']
#         ][0:10]
#     )
    # Save to file
    df.to_csv(r'hh_data.csv', index=False)

In [8]:
def analyze_df():
    """
    Load data frame and analyze results

    """

    sns.set()
    print('\n\nLoad table and analyze results:')
    df = pd.read_csv('hh_data.csv')
    print(df[df['Salary']][0:7])

    print('\nNumber of vacancies: {}'.format(df['Id'].count()))
    print('\nVacancy with max salary: ')
    print(df.iloc[df[['From', 'To']].idxmax()])
    print('\nVacancy with min salary: ')
    print(df.iloc[df[['From', 'To']].idxmin()])

    print('\nDescribe salary data frame: ')
    df_stat = df[['From', 'To']].describe().applymap(np.int32)
    print(df_stat.iloc[list(range(4)) + [-1]])

    print('\nAverage statistics (average filter for "From"-"To" parameters):')
    comb_ft = np.nanmean(df[df['Salary']][['From', 'To']].to_numpy(), axis=1)
    print('Describe salary series:')
    print('Min    : %d' % np.min(comb_ft))
    print('Max    : %d' % np.max(comb_ft))
    print('Mean   : %d' % np.mean(comb_ft))
    print('Median : %d' % np.median(comb_ft))

    print('\nMost frequently used words [Keywords]:')
    # Collect keys from df
    keys_df = df['Keys'].to_list()
    # Create a list of keys for all vacancies
    lst_keys = []
    for keys_elem in keys_df:
        for el in keys_elem[1:-1].split(', '):
            if el != '':
                lst_keys.append(re.sub('\'', '', el.lower()))
    # Unique keys and their counter
    set_keys = set(lst_keys)
    # Dict: {Key: Count}
    dct_keys = {el: lst_keys.count(el) for el in set_keys}
    # Sorted dict
    srt_keys = dict(sorted(dct_keys.items(), key=lambda x: x[1], reverse=True))
    # Return pandas series
    most_keys = pd.Series(srt_keys, name='Keys')
    print(most_keys[:12])

    print('\nMost frequently used words [Description]:')
    # Collect keys from df
    words_df = df['Description'].to_list()
    # Long string - combine descriptions
    words_ls = ' '.join(
        [re.sub(' +',
                ' ',
                re.sub(r'\d+', '', el.strip().lower())) for el in words_df]
    )
    # Find all words
    words_re = re.findall('[a-zA-Z]+', words_ls)
    # Filter words with length < 3
    words_l2 = [el for el in words_re if len(el) > 2]
    # Unique words
    words_st = set(words_l2)
    # Remove 'stop words'
    stop_words = set(nltk.corpus.stopwords.words('english'))
    # XOR for dictionary
    words_st ^= stop_words
    words_st ^= {'amp', 'quot'}
    # Dictionary - {Word: Counter}
    words_cnt = {el: words_l2.count(el) for el in words_st}
    # Pandas series
    most_words = pd.Series(
        dict(sorted(words_cnt.items(), key=lambda x: x[1], reverse=True))
    )
    print(most_words[:12])

    print('\nPlot results. Close figure box to continue...')
    fz = plt.figure('Salary plots', figsize=(12, 8), dpi=100)
    fz.add_subplot(2, 2, 1)
    plt.title('From / To: Boxplot')
    sns.boxplot(data=df[['From', 'To']].dropna(), width=0.4)
    fz.add_subplot(2, 2, 2)
    plt.title('From / To: Swarmplot')
    sns.swarmplot(data=df[['From', 'To']].dropna(), size=6)

    fz.add_subplot(2, 2, 3)
    plt.title('From: Distribution ')
    sns.distplot(df['From'].dropna(), bins=12, color='C0')
    plt.grid(False)
    plt.xlim([-50000, df['From'].max()])
    plt.yticks([], [])

    fz.add_subplot(2, 2, 4)
    plt.title('To: Distribution')
    sns.distplot(df['To'].dropna(), bins=12, color='C1')
    plt.grid(False)
    plt.xlim([-50000, df['To'].max()])
    plt.yticks([], [])
    plt.tight_layout()
    plt.show()

In [9]:
def run(search):
    """
    Main function - combine all methods together

    """
#     parser = argparse.ArgumentParser()
#     parser.add_argument('query', help='Search query (e.g. "Machine learning")')
#     parser.add_argument(
#         '--refresh',
#         help='Refresh cached data from HH API',
#         action='store_true',
#         default=False,
#     )
#     args = parser.parse_args()
    
    
    update_exchange_rates()
    print('Collect data from JSON. Create list of vacancies...')
    vac_list = get_vacancies(search)
    print('Prepare data frame...')
    prepare_df(vac_list)
    
    df = pd.read_csv("hh_data.csv")
    df.rename(columns={'Id': 'Link'}, inplace=True)
    for i in range(len(df['Link'])):
#     print(df['Id'][i])
        df['Link'][i] = 'https://kazan.hh.ru/vacancy/' + str(df['Link'][i])
    
#     return df
    df.to_excel('hh_v.xlsx', sheet_name='Angular')
    
#     analyze_df()
#     print('Done! Exit()')

In [11]:
run('Angular developer')
# ndf.to_excel('hh_vacancies.xlsx', sheet_name= 'Java', index=False)

Try to get rates from URL...
Get exchange rates: {'USD': 0.012683, 'EUR': 0.011685, 'RUR': 1}
Collect data from JSON. Create list of vacancies...


100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [00:08<00:00,  7.66it/s]


Prepare data frame...


D:\Programs\anaconda\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


parse html

In [ ]:
import requests
from bs4 import BeautifulSoup
from lxml import html

def get_html(id):
    url = 'https://vk.com/id' + id
    r = requests.get(url)
    text = r.text.encode('cp1251')
    html_page = BeautifulSoup(text)
#     print(html_page)
    return html_page

In [ ]:
arr = []
search = ['java', 'QA', 'angular', 'c#']
with pd.ExcelWriter('hh_vacancies.xlsx', engine='openpyxl') as writer:
    for el in search:
        ndf = run(el)
#         arr.append(ndf)
        ndf.to_excel(writer, sheet_name=el, index=False)
    
# print(len(arr))